In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
DIRNAME = 'smileys_dataset_all_update/'
SIZE = 32
CHANNEL = 3
files = os.listdir(DIRNAME)
N_data = len(files) * 2
train_data = np.zeros((N_data, SIZE, SIZE, CHANNEL))
for i,file in enumerate(files):
    im = Image.open(DIRNAME + file).convert('RGB').resize((SIZE, SIZE))
    train_data[i] = np.array(im).astype(np.float32).reshape(SIZE,SIZE,CHANNEL)
    train_data[i+len(files)] = train_data[i][:,::-1,:]

C:\Users\suzuk\Anaconda3\lib\site-packages\PIL\Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD

C:\Users\suzuk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
N_noise = 100
def generator_model():
    model = Sequential()
    model.add(Dense(1024, input_dim = N_noise))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128 * 8 * 8))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((128, 8, 8)))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, kernel_size = 5, strides = 1, padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, kernel_size = 5, strides = 1, padding = 'same'))
    model.add(Activation('tanh'))
    return model

In [5]:
def discriminator_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size = 5, strides = 2, padding = 'same', input_shape = (CHANNEL, SIZE, SIZE)))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, kernel_size = 5, strides = 2))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [6]:
BATCH_SIZE = 32
HALF_BATCH = int(BATCH_SIZE/2)
NUM_EPOCH = 5000
GENERATED_IMAGE_PATH = 'generated_images16/'
if not os.path.exists(GENERATED_IMAGE_PATH):
    os.mkdir(GENERATED_IMAGE_PATH)

def train():
    X_train = (train_data - 127.5) / 127.5
    X_train = X_train.reshape(N_data, CHANNEL, SIZE, SIZE)
    discriminator = discriminator_model()
    # opt_d = SGD(lr = 5e-4, momentum = 0.9, nesterov = True)
    opt_d = Adam(lr = 1e-5, beta_1 = 0.1)
    discriminator.compile(loss='binary_crossentropy', optimizer=opt_d)
    
    discriminator.trainable = False
    generator = generator_model()
    dcgan = Sequential([generator, discriminator])
    opt_g = Adam(lr = 2e-5, beta_1 = 0.5)    
    dcgan.compile(loss='binary_crossentropy', optimizer=opt_g)
    
    num_batches = int(N_data / BATCH_SIZE)
    for epoch in range(NUM_EPOCH):
        
        for index in range(num_batches):
            noise = np.array([np.random.uniform(-1, 1, N_noise) for _ in range(HALF_BATCH)])
            #noise = np.array([np.random.normal(0, 1, N_noise) for _ in range(HALF_BATCH)])
            image_batch = X_train[np.random.randint(0, N_data,HALF_BATCH)]
            generated_images = generator.predict(noise, verbose=0)
        
            if index == num_batches - 1 and epoch % 2 == 0:
                for n,image in enumerate(generated_images):
                    image = image*127.5 + 127.5
                    Image.fromarray(image.astype(np.uint8).reshape(SIZE,SIZE,CHANNEL)) \
                                                      .save(GENERATED_IMAGE_PATH+"%03d_%03d.png" % (epoch, n))
            
            d_loss_real = discriminator.train_on_batch(image_batch, np.random.uniform(1, 1,HALF_BATCH))
            d_loss_fake = discriminator.train_on_batch(generated_images, np.random.uniform(0, 0, HALF_BATCH))
            noise = np.array([np.random.uniform(-1, 1, N_noise) for _ in range(BATCH_SIZE)])
            #noise = np.array([np.random.normal(0, 1, N_noise) for _ in range(BATCH_SIZE)])
            g_loss = dcgan.train_on_batch(noise, np.random.uniform(1, 1, BATCH_SIZE))
            if index  == num_batches - 1:
                print("epoch: %d, batch: %d, g_loss: %f, d_loss_real: %f, d_loss_fake: %f" % (epoch, index, g_loss, d_loss_real, d_loss_fake))
    generator.save_weights('generator5.h5')
    discriminator.save_weights('discriminator5.h5')

In [ ]:
train()

C:\Users\suzuk\Anaconda3\lib\site-packages\keras\engine\training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
